In [2]:
import sklearn
import numpy as np
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import re
import pickle
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

In [11]:
conn = sqlite3.connect('../db.sqlite3')

In [14]:
df_raw = pd.read_sql_query("SELECT * FROM scraper_auto", conn)

In [15]:
df_raw['bron'].value_counts()

marktplaats    415
autotrack      269
autoscout      196
Name: bron, dtype: int64

In [5]:
target_names = ["bouwjaar", "kilometer_stand", "vermogen", "prijs", "is_handgeschakeld", "is_benzine", "upload_datum", "apk"]
df = df_raw[target_names]
df.describe()

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,is_benzine
count,852.000000,851.000000,852.000000,8.430000e+02,850.000000,852.000000
mean,2009.516432,169374.957697,82.863850,1.920480e+07,0.970588,0.586854
std,2.821099,64903.897649,13.232452,2.490965e+08,0.169057,0.492688
min,2005.000000,7183.000000,51.000000,1.000000e+00,0.000000,0.000000
25%,2007.000000,124985.000000,77.000000,2.249000e+03,1.000000,0.000000
50%,2009.000000,174637.000000,84.000000,2.799000e+03,1.000000,1.000000
75%,2011.000000,211476.000000,86.000000,3.950000e+03,1.000000,1.000000
max,2018.000000,368661.000000,165.000000,3.445143e+09,1.000000,1.000000


In [6]:
MAX_PRICE = 30000
MIN_PRICE = 1000
df = df[(df['prijs'] > MIN_PRICE) & (df['prijs'] < MAX_PRICE)]

MIN_BOUWJAAR = 2000
df = df[df['bouwjaar'] > MIN_BOUWJAAR]
df.describe()

MAX_VERMOGEN = 500
df = df[df['vermogen'] < MAX_VERMOGEN]
df.describe()

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,is_benzine
count,815.000000,814.000000,815.000000,815.000000,813.000000,815.000000
mean,2009.611043,166563.057740,82.788957,3580.354601,0.970480,0.588957
std,2.821839,63329.683151,12.857741,2237.300992,0.169364,0.492325
min,2005.000000,7183.000000,51.000000,1150.000000,0.000000,0.000000
25%,2007.000000,121570.250000,77.000000,2250.000000,1.000000,0.000000
50%,2010.000000,171868.000000,84.000000,2850.000000,1.000000,1.000000
75%,2011.000000,208738.500000,86.000000,3950.000000,1.000000,1.000000
max,2018.000000,367084.000000,165.000000,13900.000000,1.000000,1.000000


In [7]:
df['upload_datum'] = pd.to_datetime(df['upload_datum'])
df['apk'] = pd.to_datetime(df['apk'])

average_upload_datum = df['upload_datum'].mean().toordinal()
average_apk =df['apk'].mean().toordinal()

def upload_datum_to_ordinal(date):
    if date is pd.NaT:
        return abs(datetime.now().toordinal() - average_upload_datum)
    else:
        return abs(datetime.now().toordinal() - date.toordinal())
    
def apk_to_ordinal(date):
    if date is pd.NaT:
        return abs(datetime.now().toordinal() - average_apk)
    else:  
        return abs(datetime.now().toordinal() - date.toordinal())

df['upload_datum'] = df['upload_datum'].apply(upload_datum_to_ordinal).astype(int)
df['apk'] = df['apk'].apply(apk_to_ordinal).astype(int)

# def parse_titel(titel):
#     cleaned = re.sub('[^a-zA-Z]+', ' ', titel.lower())
#     stripped = line = re.sub(r'\b\w{1,3}\b', '', cleaned)
    
#     return stripped

# df['titel'] = df['titel'].apply(parse_titel)

In [8]:
df = df.dropna()
df

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,is_benzine,upload_datum,apk
1,2008,169504.0,78,2950.0,0.0,1,16,128
2,2007,219718.0,65,1950.0,1.0,1,11,183
3,2006,199908.0,95,1895.0,1.0,1,11,219
4,2008,138191.0,65,2740.0,1.0,1,11,381
5,2006,139057.0,65,2950.0,1.0,1,11,281
...,...,...,...,...,...,...,...,...
847,2011,219116.0,86,2700.0,1.0,0,10,63
848,2010,255591.0,86,1999.0,1.0,0,10,230
849,2008,169343.0,120,4650.0,1.0,1,10,148
850,2011,150050.0,86,2945.0,1.0,0,10,53


In [9]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
df_train.to_pickle("./train_data.pkl")
df_test.to_pickle("./test_data.pkl")